In [225]:
import json

relations = []
for line in open('/Users/alexmichailidis/gg-extraction-2022/er_orgs.jsonl', 'r'):
    relations.append(json.loads(line))

In [226]:
#relations.pop(139)#not text in an org 
len(relations)

191

In [227]:
#there was a problem in some cases during the export and in the spans (entities) it didn't get the text and the tye of entity. 
#This code makes the corrections

for r in range(len(relations)):

    for item in relations[r]["spans"]:
        if len(item) < 7:

            words=[]
            sentence=[]
            for i in range(len(relations[r]["tokens"])):
                if relations[r]["tokens"][i]["start"] == item["start"]:
                    for j in range(i,len(relations[r]["tokens"])):
                        words.append(relations[r]["tokens"][j])
            
            for word in words:
                sentence.append(word["text"])
                if item["end"]== word["end"]:
                    text=' '.join(word for word in sentence)
                    item["text"]= text
                    item["type"]= 'span'
                    break

In [228]:
#Transform based on the input desired by sample

desired_key_list = ["em1Text", "em2Text", "label"]
desired_order_list = ["start", "label", "text"]


for i in range(len(relations)-1,-1,-1):
    if relations[i]["answer"] == 'accept':
        
        #first part remove unnecessary values
        relations[i].pop("_input_hash")
        relations[i].pop("_task_hash")
        relations[i].pop("_is_binary")
        #relations[i].pop("meta")
        relations[i].pop("tokens")
        relations[i].pop("_view_id")
        relations[i].pop("_timestamp")
        relations[i].pop("answer")

        #second part transform relations
        for lst in range(len(relations[i]["relations"])):
            
            relations[i]["relations"][lst].pop('head')
            relations[i]["relations"][lst].pop('child')
            relations[i]["relations"][lst].pop('color')
            
            for item in relations[i]["spans"]:
                if relations[i]["relations"][lst]["head_span"]["start"] == item["start"]:
                    relations[i]["relations"][lst]["em2Text"] = item["text"]

                if relations[i]["relations"][lst]["child_span"]["start"] == item["start"]:
                        relations[i]["relations"][lst]["em1Text"] = item["text"]
                    

            relations[i]["relations"][lst].pop('head_span')
            relations[i]["relations"][lst].pop('child_span')
            relations[i]["relations"][lst]["label"] ='Org/Org/SubOrg'
            relations[i]["relations"][lst] = {k: relations[i]["relations"][lst][k] for k in desired_key_list}

        #third part transform spans
        j=1
        for item in relations[i]["spans"]:
                
            item.pop("token_start")
            item.pop("token_end")
            item.pop("end")
            item.pop("type")
            item["start"] = j
            j+=1
            
            item = {k: item[k] for k in desired_order_list}
            
            
        # finally rename keys
        relations[i]["sentText"] = relations[i].pop("text")
        relations[i]["entityMentions"] = relations[i].pop("spans")
        relations[i]["relationMentions"] = relations[i].pop("relations")
    
    else:
        relations.pop(i)

In [230]:
#have a look on transformed data
relations[10]

{'sentText': '"2. Η Διεύθυνση Διεθνών Σχέσεων συγκροτείται από τις ακόλουθες οργανικές μονάδες: α) Τμήμα Σχέσεων με Διεθνείς Οργανισμούς. β) Τμήμα Ευρωπαϊκών Θεμάτων, Συνεργασίας με Υπηρεσίες της Ε.Ε. και Διμερών Σχέσεων."',
 'entityMentions': [{'text': 'Διεύθυνση Διεθνών Σχέσεων',
   'start': 1,
   'label': 'ORG'},
  {'text': 'Τμήμα Σχέσεων με Διεθνείς Οργανισμούς',
   'start': 2,
   'label': 'ORG'},
  {'text': 'Τμήμα Ευρωπαϊκών Θεμάτων, Συνεργασίας με Υπηρεσίες της Ε.Ε. και Διμερών Σχέσεων',
   'start': 3,
   'label': 'ORG'}],
 'relationMentions': [{'em1Text': 'Διεύθυνση Διεθνών Σχέσεων',
   'em2Text': 'Τμήμα Σχέσεων με Διεθνείς Οργανισμούς',
   'label': 'Org/Org/SubOrg'},
  {'em1Text': 'Διεύθυνση Διεθνών Σχέσεων',
   'em2Text': 'Τμήμα Ευρωπαϊκών Θεμάτων, Συνεργασίας με Υπηρεσίες της Ε.Ε. και Διμερών Σχέσεων',
   'label': 'Org/Org/SubOrg'}]}

In [231]:
#save each in a row

filename = '/Users/alexmichailidis/gg-extraction-2022/data/ER/er-four-orgs.json'

with open(filename, "w") as fp: 
    for my_dict in relations:
        if (my_dict!=relations[-1]):
            fp.write(json.dumps(my_dict, ensure_ascii=False)+'\n')
        else:
            fp.write(json.dumps(my_dict, ensure_ascii=False)) 

In [ ]:
#alternative way not in a row
filename = '/Users/alexmichailidis/gg-extraction-2022/data/er_test.json'

with open(filename, "w") as outfile:  # also, tried mode="rb"
    json.dump(relations, outfile, ensure_ascii=False, indent=2)